# Check missings:  images, videos and annotations
version: 1

info: 


author: nuno costa

In [58]:
from annotate_v5 import *
import platform 
import numpy as np
import pandas as pd
from IPython.display import Image, display
import copy
import os
import random

In [81]:
dataset_root="TAO/TAO_DIR"

In [82]:
#Define root dir dependent on OS
rdir_dsets='/home/administrator/Z/Datasets/External Datasets/' #WARNING: DATASETS ROOT is OK?
rdir='/home/administrator/Z/Datasets/External Datasets/'+ dataset_root +'/'
print('OS: {}'.format(platform.platform()))
print('root datasets dir: {}'.format(rdir_dsets))
print('root dir: {}'.format(rdir))

OS: Linux-5.4.0-65-generic-x86_64-with-glibc2.10
root datasets dir: /home/administrator/Z/Datasets/External Datasets/
root dir: /home/administrator/Z/Datasets/External Datasets/TAO/TAO_DIR/


In [83]:
#jsonfile
injsonfile_rdir="annotations/validation" #"split_mola_fix_equal/test"
molajson =  json.load(open(rdir+injsonfile_rdir+'.json'))
for k in molajson:
    print(k, len(molajson[k]))

videos 988
annotations 113112
tracks 5485
images 36375
info 6
categories 1230
licenses 1


In [84]:
#jsonfile
injsonfile_rdir="annotations/train" #"split_mola_fix_equal/test"
molajson =  json.load(open(rdir+injsonfile_rdir+'.json'))
for k in molajson:
    print(k, len(molajson[k]))

videos 500
annotations 54639
tracks 2647
images 18274
info 6
categories 1230
licenses 1


In [85]:
#jsonfile
injsonfile_rdir="annotations/fulltao" #"split_mola_fix_equal/test"
molajson =  json.load(open(rdir+injsonfile_rdir+'.json'))
for k in molajson:
    print(k, len(molajson[k]))

videos 1488
annotations 167751
tracks 8132
images 54649
info 6
categories 1230
licenses 1


# A. Verify using dataset toolkit

In [87]:
#TAO
if (dataset_root.upper().find("TAO")>-1):
    #https://github.com/TAO-Dataset/tao
    #https://github.com/TAO-Dataset/tao/blob/master/docs/download.md
    TAO_ROOT=rdir#'"'+rdir+'"'
    TAO_SCRIPT='"/home/administrator/Z/Datasets/External Datasets/TAO/tao/scripts/download/verify.py"'
    PASSFILE="/home/administrator/Desktop/remote.txt"
    !sudo -S python $TAO_SCRIPT $TAO_ROOT --split train <$PASSFILE

[sudo] password for administrator:   File "/home/administrator/Z/Datasets/External Datasets/TAO/tao/scripts/download/verify.py", line 28
    ann_path = args.root / f'annotations/{args.split}.json'
                                                          ^
SyntaxError: invalid syntax


## 1. Import
#### #NOTE: work with ids and index so you can use numpy for faster operations

In [88]:
# datasets name and id
dset_l=[]
dset_l_id=[]
try:
    for d in molajson['datasets']:
        dset_l.append(d['name'])
        dset_l_id.append(d['id'])
except: #manually add for example for only COCO
    dset_l=['TAO']
    dset_l_id=[1]
print(dset_l, dset_l_id)

['TAO'] [1]


In [89]:
# categories name and id
cat_l=[]
cat_l_id=[]
cat_l_dset=[]
for c in molajson['categories']:
    cat_l.append(c['name'])
    cat_l_id.append(c['id'])
    try:
        cat_l_dset.append(dset_l[c['dataset']-1]) # dset_l index is same as id-1
    except:
        cat_l_dset.append(dset_l[0])
#print(cat_l_id)

In [90]:
# images filepath and id
img_l=[]
img_l_id=[]
for c in molajson['images']:
    img_l.append(c['file_name'])
    img_l_id.append(c['id'])

In [91]:
# videos filepath and id
try:
    vid_l=[]
    vid_l_id=[]
    for c in molajson['videos']:
        vid_l.append(c['name'])
        vid_l_id.append(c['id'])
except Exception as e:
    print(e)
    print("No video")

In [92]:
# tracks 
try:
    track_l_id=[]
    track_l_vidid=[]
    for c in molajson['tracks']:
        track_l_id.append(c['id'])
        track_l_vidid.append(c['video_id'])
except Exception as e:
    print(e)
    print("No video")

In [93]:
# annotations category_id, image_id, bbox, and dataset
ann_catid=[]
ann_imgid=[]
ann_bbox=[]
ann_dset=[]
ann_trackid=[]
for an in tqdm(molajson['annotations']):
    ann_catid.append(an['category_id'])
    ann_imgid.append(an['image_id'])
    ann_bbox.append(an['bbox'])
    try: 
        ann_trackid.append(an['track_id'])
    except:
        pass
    try:
        ann_dset.append(an['dataset'])
    except:
        ann_dset.append(dset_l_id[0])

100%|██████████| 167751/167751 [00:00<00:00, 745195.50it/s]


## 2. CHECK IMAGES

In [99]:
img_missing_l=[]
for img in tqdm(img_l):
    src = os.path.join(rdir_dsets, img)
    #if (dataset_root.upper().find("TAO")>-1): src = os.path.join(rdir,"frames", img)
    if not os.path.exists(src): 
        img_missing_l.append(img)
print(src)
print("\n>>IMAGES MISSING: ", len(img_missing_l))

100%|██████████| 54649/54649 [00:11<00:00, 4612.26it/s] 

/home/administrator/Z/Datasets/External Datasets/TAO/TAO_DIR/frames/val/Charades/2KGV3/frame0751.jpg

>>IMAGES MISSING:  19382


### 2.1 CHECK ANNOTATIONS (with missing images, that have good bounding box and are are not "iscrowd")

In [101]:
# Create image dict {id: image}
images = {'%g' % x['id']: x for x in molajson['images']}

# annotations category_id
new_annotations=[]
img_missing_l=[]
ann_missing_l=[]
for an in tqdm(molajson['annotations']):
    try:
        if an['iscrowd']: continue
    except:
        print('missing "iscrowd" key')
    img = images['%g' % an['image_id']]
    h, w, f = img['height'], img['width'], img['file_name']


    # The Labelbox bounding box format is [top left x, top left y, width, height]
    box = np.array(an['bbox'], dtype=np.float64)
    box[:2] += box[2:] / 2  # xy top-left corner to center
    box[[0, 2]] /= w  # normalize x
    box[[1, 3]] /= h  # normalize y

    if (box[2] > 0.) and (box[3] > 0.):  # if w > 0 and h > 0
        src = os.path.join(rdir_dsets, f)
        #if (dataset_root.upper().find("TAO")>-1): src = os.path.join(rdir,"frames", f)
        if os.path.exists(src): new_annotations.append(an)
        else: 
            img_missing_l.append(img)
            ann_missing_l.append(an)
            

print(len(molajson['annotations']))
print(len(new_annotations))
print("\n>>ANNOTATIONS W/ IMAGES MISSING: ", len(img_missing_l))

100%|██████████| 167751/167751 [00:11<00:00, 14252.38it/s]

167751
100066

>>ANNOTATIONS W/ IMAGES MISSING:  67685


### 2.2 CHECK Classes|categories (annotations and images for each class w/ EXCEL report) 

In [102]:
#checks #TODO: SORT alphabetically
checks_l=[]
checks_l_catid=[]
checks_l_catdset=[]
check_method="all_cats"
if check_method=="all_cats": #Do for all category names, even with equal 
    checks_l=cat_l
    checks_l_catid=[[id] for id in cat_l_id]
    checks_l_catdset=[[dset] for dset in cat_l_dset]
    

print(checks_l[0:5])
print(checks_l_catid[0:5])
print(checks_l_catdset[0:5])
print(len(checks_l))
print(len(checks_l_catid))
print(len(checks_l_catdset))

['acorn', 'aerosol_can', 'air_conditioner', 'airplane', 'alarm_clock']
[[1], [2], [3], [4], [5]]
[['TAO'], ['TAO'], ['TAO'], ['TAO'], ['TAO']]
1230
1230
1230


In [103]:
# get annotations checks
ann_catid_np=np.array(ann_catid)
ann_imgid_np=np.array(ann_imgid)
ann_bbox_np=np.array(ann_bbox)
ann_dset_np=np.array(ann_dset)
checks_l_imgid=[]
checks_l_bbox=[]
checks_l_dset=[]
for catids in tqdm(checks_l_catid):
    l_imgid=[]
    l_bbox=[]
    l_dset=[]
    for catid in catids:
        ann_idx = np.where(ann_catid_np==catid)[0].tolist() #annotation index of ids
        l_imgid.append(ann_imgid_np[ann_idx].tolist())
        l_bbox.append(ann_bbox_np[ann_idx].tolist())
        l_dset.append(ann_dset_np[ann_idx].tolist())
    checks_l_imgid.append(l_imgid)
    checks_l_bbox.append(l_bbox)
    checks_l_dset.append(l_dset)

100%|██████████| 1230/1230 [00:00<00:00, 4337.52it/s]


In [104]:
#INIT VARS
method="save_images" #"save_images": to save new images and excel for manual inspection; "": already saved don't need to repeat and the excel are
datadir="checks/"+injsonfile_rdir+"/" #root folder to save check method . #WARNING checks/original json that was used to save images and excel
folder=check_method+'/' #folder to save images and exel 
showimage=False #show images
startidx=0 # start index of image to save from each dataset
imgnr=3 # total number of images to save from each dataset
imgstep='random' # step between images: int | 'random' - int steps between images; 'rand' gets random list
#paths
path=os.path.join(rdir,datadir,folder) #path to folder
assure_path_exists(path)
excelpath=path+check_method+"_report_v1.xlsx"#path+check_method+"_report.xlsx"#path to excel

In [106]:
#METHODS
if method=="save_images": # save images and excel to folder for manual edit
    df=pd.DataFrame({'checks_l': checks_l,'checks_l_catid': checks_l_catid, 'checks_l_catdset': checks_l_catdset })
    df['annotations_missing'] = np.empty((len(df), 0)).tolist()
    df['images_missing'] = np.empty((len(df), 0)).tolist()
    #save image for each check
    for i, check in enumerate(tqdm(checks_l)): #run for each check category
        firstclass=check
        if isinstance(firstclass, list): firstclass=firstclass[0] #first class
        print('\n>> '+firstclass+'...') #class
        classpath=os.path.join(path, firstclass) # path to folder for images of  firstclass
        classpath=parse_path(classpath)+'/' #make it a folder
        assure_path_exists(classpath)
        rdir_path=rdir_dsets
        #if (dataset_root.upper().find("TAO")>-1): rdir_path = os.path.join(rdir,"frames/")
        df=save_imgs(df, rdir_path, classpath, i, dset_l, checks_l, checks_l_catid, checks_l_bbox, checks_l_dset,
              checks_l_imgid, img_l, img_l_id, startidx=startidx, imgnr=imgnr, imgstep=imgstep, showimage=showimage)    
    df.to_excel(excelpath, index=False)   

>>> Finding TAO aerosol_can...: 100%|██████████| 40/40 [00:00<00:00, 9716.91it/s]

>>> Finding TAO aerosol_can...: 100%|██████████| 40/40 [00:00<00:00, 12639.16it/s]

>>> Finding TAO aerosol_can...: 100%|██████████| 40/40 [00:00<00:00, 10904.21it/s]



>> acorn...
>>>> #WARNING acorn is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []

>> aerosol_can...
TAO
[Errno 2] No such file or directory: '/home/administrator/Z/Datasets/External Datasets/TAO/TAO_DIR/frames/train/HACS/Painting_furniture_v_xNxxM-OOMfw_scene_0_0-1910/frame0511.jpg'
>>>> #WARNING: No file found for: TAO aerosol_can...
[Errno 2] No such file or directory: '/home/administrator/Z/Datasets/External Datasets/TAO/TAO_DIR/frames/train/HACS/Painting_furniture_v_xNxxM-OOMfw_scene_0_0-1910/frame0391.jpg'
>>>> #WARNING: No file found for: TAO aerosol_can...
[Errno 2] No such file or directory: '/home/administrator/Z/Datasets/External Datasets/TAO/TAO_DIR/frames/train/HACS/Painting_furniture_v_xNxxM-OOMfw_scene_0_0-1910/frame1051.jpg'
>>>> #WARNING: No file found for: TAO aerosol_can...

>> air_conditioner...
>>>> #WARNING air_conditioner is missing from json annotations
annotations datasets:  []
annotations imgids:  []
anno

  1%|▏         | 16/1230 [00:00<00:36, 33.71it/s]


>> alarm_clock...
>>>> #WARNING alarm_clock is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []

>> alcohol...
>>>> #WARNING alcohol is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []

>> alligator...
>>>> #WARNING alligator is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []

>> almond...
>>>> #WARNING almond is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []

>> ambulance...
>>>> #WARNING ambulance is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []

>> amplifier...
>>>> #WARNING amplifier is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []

>> anklet...
>>>> #WARNING anklet is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations 

>>> Finding TAO atomizer...: 100%|██████████| 24/24 [00:00<00:00, 2250.06it/s]

>>> Finding TAO atomizer...: 100%|██████████| 24/24 [00:00<00:00, 3379.55it/s]

>>> Finding TAO atomizer...: 100%|██████████| 24/24 [00:00<00:00, 3264.26it/s]

>>> Finding TAO ax...: 100%|██████████| 174/174 [00:00<00:00, 4018.09it/s]

>>> Finding TAO ax...: 100%|██████████| 174/174 [00:00<00:00, 4135.81it/s]

  3%|▎         | 31/1230 [00:01<00:36, 32.47it/s]

...


>> wing_chair...
>>>> #WARNING wing_chair is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []

>> blinder_(for_horses)...
>>>> #WARNING blinder_(for_horses) is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []

>> wok...
>>>> #WARNING wok is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []

>> wolf...
>>>> #WARNING wolf is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []

>> wooden_spoon...
TAO


 99%|█████████▉| 1220/1230 [01:37<00:00, 20.39it/s]


>> wreath...
>>>> #WARNING wreath is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []

>> wrench...
>>>> #WARNING wrench is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []

>> wristband...
>>>> #WARNING wristband is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []

>> wristlet...
>>>> #WARNING wristlet is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []

>> yacht...
>>>> #WARNING yacht is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []

>> yak...
>>>> #WARNING yak is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []

>> yogurt...
>>>> #WARNING yogurt is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []

>> yoke_(animal_e

100%|██████████| 1230/1230 [01:38<00:00, 12.53it/s]


>> zucchini...
>>>> #WARNING zucchini is missing from json annotations
annotations datasets:  []
annotations imgids:  []
annotations bbox:  []


In [107]:
#IMPORT EXCEL MANUAL EDIT #WARNING: CHECK EXCEL FIRST (#NOTE: donte use classes with missing annotations and images)
df=pd.read_excel(excelpath)
display(df)        

,checks_l,checks_l_catid,checks_l_catdset,annotations_missing,images_missing
0,acorn,[1],['TAO'],[1],[0]
1,aerosol_can,[2],['TAO'],[0],[1]
2,air_conditioner,[3],['TAO'],[1],[0]
3,airplane,[4],['TAO'],[0],[0]
4,alarm_clock,[5],['TAO'],[1],[0]
...,...,...,...,...,...
1225,yak,[1226],['TAO'],[1],[0]
1226,yogurt,[1227],['TAO'],[1],[0]
1227,yoke_(animal_equipment),[1228],['TAO'],[1],[0]
1228,zebra,[1229],['TAO'],[0],[0]


## 3. CHECK VIDEOS

In [109]:
vid_missing_l=[]
for vid in vid_l:
    src = os.path.join(rdir_dsets, vid)
    #if (dataset_root.upper().find("TAO")>-1): src = os.path.join(rdir,"videos", vid)
    if not os.path.exists(src): 
        vid_missing_l.append(vid)
print(src)
print("\n>>VIDEOS MISSING: ", len(vid_missing_l))

/home/administrator/Z/Datasets/External Datasets/val/Charades/2KGV3

>>VIDEOS MISSING:  1488


### 3.1 CHECK ANNOTATIONS (with missing ivideos)

In [111]:
# Create image dict {id: image}
videos = {'%g' % x['id']: x for x in molajson['videos']}
tracks = {'%g' % x['id']: x for x in molajson['tracks']}
# annotations category_id
new_annotations=[]
vid_missing_l=[]
ann_missing_l=[]
for an in tqdm(molajson['annotations']):
    try:
        if an['iscrowd']: continue
    except:
        print('missing "iscrowd" key')
    
    vid = videos['%g' % tracks['%g' % an['track_id']]["video_id"]]
    h, w, f = vid['height'], vid['width'], vid['name']


    # The Labelbox bounding box format is [top left x, top left y, width, height]
    box = np.array(an['bbox'], dtype=np.float64)
    box[:2] += box[2:] / 2  # xy top-left corner to center
    box[[0, 2]] /= w  # normalize x
    box[[1, 3]] /= h  # normalize y

    if (box[2] > 0.) and (box[3] > 0.):  # if w > 0 and h > 0
        src = os.path.join(rdir, f)
        if (dataset_root.upper().find("TAO")>-1): src = os.path.join(rdir,"videos", f)
        if os.path.exists(src): new_annotations.append(an)
        else: 
            vid_missing_l.append(img)
            ann_missing_l.append(an)
            

print(len(molajson['annotations']))
print(len(new_annotations))
print("\n>>ANNOTATIONS W/ VIDEOS MISSING: ", len(vid_missing_l))

100%|██████████| 167751/167751 [00:03<00:00, 44213.01it/s]

167751
0

>>ANNOTATIONS W/ VIDEOS MISSING:  167751
